In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.optim import SGD
from timeit import default_timer as timer

device = "cuda" if torch.cuda.is_available() else "cpu"
print( f"device: {device}" )


# Umschalten zwischen Colab oder lokaler Installation
USING_COLAB = True

if USING_COLAB:
  from google.colab import drive
  from google.colab.patches import cv2_imshow
  drive.mount('/content/drive')

Download and load the training data

In [ ]:
train_set = datasets.MNIST('data/', download=True, train=True)
train_images = train_set.data
train_targets = train_set.targets

In [ ]:
test_set = datasets.MNIST('data/', download=True, train=False)
test_images = test_set.data
test_targets = test_set.targets

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()/255
        x = x.view(-1,28*28)
        self.x, self.y = x, y
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)

In [ ]:
def get_data():
    train = MNISTDataset(train_images, train_targets)
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test = MNISTDataset(test_images, test_targets)
    test_dl = DataLoader(test, batch_size=len(test_images), shuffle=True)
    return train_dl, test_dl

Modell (KNN) definieren mit beliebig vielen Schichten, die jeweils variable Anzahl Neuronen beinhalten. Wir beginnen hier immer mit 28x28 Eingabe-Neuronen und müssen am Ende immer auf 10 Ausgabe-Neuronen kommen

In [ ]:
def get_model():
  model = nn.Sequential(
    nn.Linear(28 * 28, 30),
    nn.Tanh(),
    nn.Linear(30, 20),
    nn.Tanh(),
    nn.Linear(20, 10)
    ).to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = SGD(model.parameters(), lr=1e-2)
  return model, loss_fn, optimizer

In [ ]:
def init_weights(m):
  if type(m) == nn.Linear:
    #m.weight.data.fill_(1)
    #m.weight.data.uniform_(-0.1, 0.1)
    m.weight.data.normal_(0.0, 0.1)
    if m.bias is not None:
      m.bias.data.fill_(0)

In [ ]:
def train_batch(x, y, model, opt, loss_fn):
  model.train()
  prediction = model(x)
  batch_loss = loss_fn(prediction, y)
  batch_loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  return batch_loss.item()

In [ ]:
def accuracy(x, y, model):
  model.eval()
  with torch.no_grad():
    prediction = model(x)
  max_values, argmaxes = prediction.max(-1)
  is_correct = argmaxes == y
  return is_correct.cpu().numpy().tolist()

In [ ]:
def loss(x, y, model, loss_fn):
  model.eval()
  with torch.no_grad():
    prediction = model(x)
    loss = loss_fn(prediction, y)
  return loss.item()

In [ ]:
train_dl, test_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
#----------------------------------------------
# Training >>>
#
print('Starting training...')

model.apply(init_weights)  # hier werden die initialen Gewichte des Netzes zufällig gesetzt

epochs = 50

arrPlotX = []
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []
for epoch in range(epochs):
  timeBeginEpoch = timer()
  train_epoch_losses, train_epoch_accuracies = [], []
  
  for ix, batch in enumerate(iter(train_dl)):
    x, y = batch
    batch_loss = train_batch(x, y, model, optimizer, loss_fn)
    train_epoch_losses.append(batch_loss)
    is_correct = accuracy(x, y, model)
    train_epoch_accuracies.extend(is_correct)

  train_epoch_loss = np.array(train_epoch_losses).mean()
  train_epoch_accuracy = np.mean(train_epoch_accuracies)
 
  for ix, batch in enumerate(iter(test_dl)):
    x, y = batch
    val_is_correct = accuracy(x, y, model)
    validation_loss = loss(x, y, model, loss_fn)
    
  val_epoch_accuracy = np.mean(val_is_correct)
  arrPlotX.append(epoch)
  train_losses.append(train_epoch_loss)
  train_accuracies.append(train_epoch_accuracy)
  test_losses.append(validation_loss)
  test_accuracies.append(val_epoch_accuracy)
  timeEndEpoch = timer()
  print( f"epoch: {epoch}  train_acc: {100 * train_epoch_accuracy:.2f}%  test_acc: {100 * val_epoch_accuracy:.2f}%  took {timeEndEpoch-timeBeginEpoch:.1f}s" )   

In [ ]:
if USING_COLAB:
  torch.save(model.state_dict(), '/content/drive/My Drive/ColabNotebooks/results/nnMnist_exp01.pt')
else:
  torch.save(model.state_dict(), 'nnMnist_exp01.pt')

In [ ]:
plt.plot(arrPlotX, train_accuracies)
plt.plot(arrPlotX, test_accuracies)

if USING_COLAB:
  plt.savefig('/content/drive/My Drive/ColabNotebooks/results/accuracies_exp0.png')
else:
  plt.savefig('accuracies_exp0.png')

In [ ]:
plt.plot(arrPlotX, train_losses)
plt.plot(arrPlotX, test_losses)

if USING_COLAB:
  plt.savefig('/content/drive/My Drive/ColabNotebooks/results/losses_exp0.png')
else:
  plt.savefig('losses_exp0.png')